### Условие задания

975 родителей участвовало в программе обучения воспитанию. Было проведено три опроса, в ходе которых родители отвечали на вопрос: «За последние несколько недель обращались ли дети к вам с проблемой или вопросом, который их беспокоил?» Первый опрос был проведён до начала обучения, второй - сразу после, и третий - по прошествии 6-8 недель после окончания обучения. Известен также уровень образования родителя.

Задача: стали ли родители больше общаться с детьми в результате обучения? Проанализировать с учётом уровня образования родителей

In [171]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
from pandas import DataFrame as df
import numpy as np


from scipy.stats import norm
from scipy.stats import chi2_contingency

In [164]:
data['Time1'] = (data['Time1'] == 'Yes').astype(int)
data['Time2'] = (data['Time2'] == 'Yes').astype(int)
data['Time3'] = (data['Time3'] == 'Yes').astype(int)

In [165]:
data

,Time1,Time2,Time3,11+,<11
0,1,1,1,135,95
1,1,1,0,26,32
2,1,0,1,30,33
3,1,0,0,32,30
4,0,1,1,79,74
5,0,1,0,29,35
6,0,0,1,65,57
7,0,0,0,94,129


Проверим гипотезу о том, что результаты опросов не зависят от уровня образования родителей. Рассмотрим таблицу сопряженности для двух переменных: первая ($x_1$) представляет собой совокупность результатов трех опросов и определена на множестве булевых векторов размерности 3, вторая ($x_2$) - булева переменная, равная True, если у родителей есть высшее образование, и False иначе. Тогда мы хотим проверить, верно ли, что $P(x_1 | x_2 = 0) = P(x_1 | x_2 = 1) = P(x_1)$ или $P(x_1, x_2) = P(x_1 | x_2) P(x_2) = P(x_1) P(x_2)$, то есть что переменные $x_1$ и $x_2$ независимы.

У нас есть выборки $X_1 = (x_{1 1}, \dots, x_{1 n})$ и $X_2 = (x_{2 1}, \dots, x_{2 n})$, состоящие из значений переменных $x_1$ и $x_2$ для $n$ объектов - людей, посещавших курсы. Так как значения переменных приведены для одних и тех же объектов, то выборки связанные. Так как суммарный объем выборок фиксирован (и равен числу людей, записавшихся на курс), то данные описываются мультиномиальной моделью. Рассмотрим, какие критерии можно использовать для проверки гипотез о таких данных.

Будем проверять нулевую гипотезу о независимости переменных $x_1$ и $x_2$ против общей альтернативы: гипотеза $H_0$ неверна. Так как таблица сопряженности имеет большой размер, и значения в ячейках достаточно большие, то точный тест Фишера потребует слишком больших вычислительных затрат. Более того, значения достаточно большие, чтобы обеспечить адекватную работу приближенных методов поиска достигаемого уровня значимости. 

Известно, что G-критерий является тестом отношения правдоподобия для мультиномиального распределения. Действительно, пусть $\theta \in \Theta$ и пусть нулевая гипотеза $\theta \in \Theta_0$ проверяется против общей альтернативы $\theta \in \Theta \setminus \Theta_0$. Статистика теста отношения правдоподобия в общем виде:

$$ \lambda_{L R} = -2 ln \Big( \frac{sup_{\theta \in \Theta_0} L(\theta)}{sup_{\theta \in \Theta} L(\theta)} \Big)$$

При условии, что нулевая гипотеза верна, распредение статистики стремится к распределению хи-квадрат.

В случае мультиномиального распределения с $K_1 \cdot K_2$ классами $L(p) = const  \prod_{i = 1}^{K_1} \prod_{j = 1}^{K_2} p_{i j}^{n_{i j}}$, где $p = (p_{1 1}, \dots, p_{K_1 K_2})$ - вектор параметров распределения, а $n_{1 1}, \dots, n_{K_1 K_2}$ - количество элементов из соответствующих классов в выборке. В случае, если нулевая гипотеза верна, максимум правдоподобия достигается при $p_{i j} = \frac{n_{i +} n_{+ j}}{n}$. В случае, когда ограничений на параметры нет, максимум достигается при $p_{i j} = n_{i j}$. Тогда статистика принимает вид

$$ \lambda_{L R} = -2 ln \Big( \frac{ const \prod_{i = 1}^{K_1} \prod_{j = 1}^{K_2} (\frac{n_{i+} n_{+j}}{n})^{n_{i j}}}{const \prod_{i = 1}^{K_1} \prod_{j = 1}^{K_2} n_{i j}^{n_{i j}}} \Big) = -2 \sum_{i = 1}^{K_1} \sum_{j = 1}^{K_2} n_{i j} ln( \frac{n_{i+} n_{+j}}{n_{i j} n} \Big) = 2 \sum_{i = 1}^{K_1} \sum_{j = 1}^{K_2} n_{i j} ln( \frac{n_{i j} n}{n_{i+} n_{+j}} \Big)$$

что совпадает с формулой для статистики G-критерия.

Покажем, что статистика критерия хи-квадрат аппроксимирует статистику G-критерия с использованием разложения по Тейлору логарифма. Обозначим $n_{i j} = o_{i j}, \; \frac{n_{i+} n_{+j}}{n} = e_{i j}$. Пусть также $o_{i j} = e_{i j} + \delta_{i j}$,  Тогда Статистика G-критерия имеет следующий вид:
$$ G^2 = 2 \sum_{i = 1}^{K_1} \sum_{j = 1}^{K_2} (e_{i j} + \delta_{i j}) log(\frac{e_{i j} + \delta_{i j}}{e_{i j}}) = 2 \sum_{i = 1}^{K_1} \sum_{j = 1}^{K_2} (e_{i j} + \delta_{i j}) log(1 + \frac{\delta_{i j}}{e_{i j}}) $$

По формуле Тейлора, $log(1 + x) = x - \frac{x^2}{2} + o(x^3)$. Тогда

$$G^2 = 2 \sum_{i = 1}^{K_1} \sum_{j = 1}^{K_2} (e_{i j} + \delta_{i j}) \Big( \frac{\delta_{i j}}{e_{i j}} - \frac{1}{2} \frac{\delta^2_{i j}}{e^2_{i j}}  + o(\delta^3_{i j}) \Big) = 2 \sum_{i = 1}^{K_1} \sum_{j = 1}^{K_2} \delta_{i j} + 2 \sum_{i = 1}^{K_1} \sum_{j = 1}^{K_2} \Big(\frac{1}{2} \frac{\delta^2_{i j}}{e_{i j}} + o(\delta^3_{i j}) \Big)  $$

Заметим, что $\sum_{i = 1}^{K_1} \sum_{j = 1}^{K_2} o_{i j} = \sum_{i = 1}^{K_1} \sum_{j = 1}^{K_2} e_{i j} = n$, тогда $\sum_{i = 1}^{K_1} \sum_{j = 1}^{K_2} \delta_{i j} = 0$. Получаем, что

$$G^2 = \sum_{i = 1}^{K_1} \sum_{j = 1}^{K_2} \Big(\frac{\delta^2_{i j}}{e_{i j}} + o(\delta^3_{i j}) \Big) $$

Вернувшись к первоначальным обозначениям, получаем 

$$ G^2 \simeq \sum_{i = 1}^{K_1} \sum_{j = 1}^{K_2}  \frac{(n_{i j} - \frac{n_{i+} n_{+j}}{n})^2}{\frac{n_{i+} n_{+j}}{n}} = n \Big(  \sum_{i = 1}^{K_1} \sum_{j = 1}^{K_2} \frac{n_{i j} n}{n_{i+} n_{+j}} - 1 \Big)$$

Это в точности совпадает со статистикой критерия хи-квадрат. Таким образом, можно утверждать, что при выполнении ряда условий, обеспечивающих хорошее приближение разложением до второго порядка, результаты применения G-критерия и критерия чи=квадрат имеют незначительные различия. В остальных случаях распределение статистики G-критерия лучше аппроксимирует нулевое распределение, и лучше использовать его.

Формально проверим условия применимости критерия хи-квадрат для наших данных.

In [298]:
n = np.sum(d)
n * (np.sum(d ** 2 / (np.sum(d, axis=0)[np.newaxis, :] * np.sum(d, axis=1)[:, np.newaxis])) - 1)

14.503087449165847

In [266]:
def check_chi_squared_cond(table):
    n = np.sum(table)
    counter = np.sum(np.sum(table, axis=0)[np.newaxis, :] * np.sum(table, axis=1)[:, np.newaxis] / n < 5)
    return n > 40 and counter < 0.2 * (table.shape[0] * table.shape[1])

In [267]:
check_chi_squared_cond(data.loc[:, ['11+','<11']].values)

True

Условия выполняются, поэтому различия между результатами использования критериев должны быть незначительными. Вычислим значение достигаемого уровня значимости.

In [268]:
print('G test: p-value =', chi2_contingency(data.loc[:, ['11+','<11']].values, lambda_="log-likelihood")[1])
print('chi-square test: p-value =', chi2_contingency(data.loc[:, ['11+','<11']].values)[1])

G test: p-value = 0.04202158856730284
chi-square test: p-value = 0.04292364379445899


Оба критерия позволяют отвергнуть нулевую гипотезу на уровне значимости 0.05. То есть существует зависимость результатов опросов от уровня образования родителей. 

Но, на самом деле, нас интересуют не сами результаты, а то, как они изменились после прохождения курса. Введем новые категориальные переменные, принимающие значения из {-1, 0, 1}, описывающие изменения между результатами пары опросов. Значение 1 соответствует изменению в лучшую сторону, -1 - в худшую, 0 - отсутствию изменений.

In [270]:
data['diff_12'] = data['Time2'] - data['Time1']
data['diff_23'] = data['Time3'] - data['Time2']
data['diff_13'] = data['Time3'] - data['Time1']

In [279]:
data_diff = data.groupby(by=['diff_12', 'diff_23', 'diff_13']).agg({'11+':'sum','<11':'sum'}).reset_index()
data_diff

,diff_12,diff_23,diff_13,11+,<11
0,-1,0,-1,32,30
1,-1,1,0,30,33
2,0,-1,-1,26,32
3,0,0,0,229,224
4,0,1,1,65,57
5,1,-1,0,29,35
6,1,0,1,79,74


Рассмотрим переменную $x'_1$, представляющую собой попарные изменения в результатах опросов и определенную на $\{-1, 0, 1\}^3$, и переменную $x_2$ - булеву переменную, равную True, если у родителей есть высшее образование, и False иначе. Проверим нулевую гипотезу о том, что переменные $x'_1$ и $x_2$ независимы, против общей альтернативы.

Аналогично проверим условия применимости критерия хи-квадрат

In [282]:
check_chi_squared_cond(data_diff.loc[:, ['11+','<11']].values)

True

Критерий применим. Вычислим значение достигаемого уровня значимости.

In [284]:
print('G test: p-value =', chi2_contingency(data_diff.loc[:, ['11+','<11']].values, lambda_="log-likelihood")[1])
print('chi-square test: p-value =', chi2_contingency(data_diff.loc[:, ['11+','<11']].values)[1])

G test: p-value = 0.9092595516172199
chi-square test: p-value = 0.9094876709806509


На уровне значимости 0.05 нулевая гипотеза не отвергается. Мы не можем утверждать, что есть статистически значимая зависимость между изменениями в результатах опросов и уровнем образования родителей.

Для ответа на вопрос, стали ли родители больше общаться с детьми после прохождения курса, будем сравнивать вероятность положительного ответа на вопрос «За последние несколько недель обращались ли дети к вам с проблемоиB или вопросом, который их беспокоил?» до и после прохождения курса. Для этого будем использовать z-критерий для разности долей для связанных выборок. Критерий учитывает только суммарное число объектов и число объектов, для которых признак изменился. В нашем случае важны не сами результаты опросов, а то, как они изменились. 

А, как показали эксперименты, для изменений в результатах опросов статистически значимая зависимость от уровня образования родителей не выявлена. Тогда группы можно объединить.

In [287]:
data_summed = data.loc[:, ['Time1','Time2', 'Time3']]
data_summed['count'] = data['11+'] + data['<11']
data_summed

,Time1,Time2,Time3,count
0,1,1,1,230
1,1,1,0,58
2,1,0,1,63
3,1,0,0,62
4,0,1,1,153
5,0,1,0,64
6,0,0,1,122
7,0,0,0,223


In [288]:
def z_test(table, alternative='two-sided'):
    """
    z-критерий для разности долей для связанных выборок
    H0: p_1 == p_2
    H1: p_1 < != > p_2
    
    table: array 2x2 of counts
    
    """
    g = table[0, 1]
    f = table[1, 0]
    n = np.sum(table)
    Z = (f - g) / np.sqrt(f + g - (f - g) ** 2 / n)
    
    if alternative == 'left':
        return norm.cdf(Z)
    elif alternative == 'right':
        return 1 - norm.cdf(Z)
    elif alternative == 'two-sided':
        return 2 * (1 - norm.cdf(np.abs(Z)))
    

Для каждой пары опросов проверим нулевую гипотезу о том, что вероятности положительных ответов равны, против односторонней альтернативы: вероятность положительного ответа во втором опросе увеличилась. 

In [296]:
table_12 = df.pivot(data_summed.groupby(by=['Time1', 'Time2'])[['count']].sum().reset_index(), 
         index='Time1', columns='Time2', values='count')
print('p-value =', z_test(table_12.values, alternative='left'))
table_12

p-value = 2.3379454348556563e-07


Time2,0,1
Time1,,
0,345,217
1,125,288


In [293]:
table_23 = df.pivot(data_summed.groupby(by=['Time2', 'Time3'])[['count']].sum().reset_index(), 
         index='Time2', columns='Time3', values='count')
print('p-value =', z_test(table_23.values, alternative='left'))
table_23

p-value = 0.0001474848267757306


Time3,0,1
Time2,,
0,285,185
1,122,383


In [294]:
table_13 = df.pivot(data_summed.groupby(by=['Time1', 'Time3'])[['count']].sum().reset_index(), 
         index='Time1', columns='Time3', values='count')
print('p-value =', z_test(table_13.values, alternative='left'))
table_13

p-value = 4.001318539159966e-16


Time3,0,1
Time1,,
0,287,275
1,120,293


На уровне значимости 0.05 все гипотезы уверенно отвеграются. Поэтому на основе имеющихся данных можно утверждать, что в результате обучения родители, действительно, стали больше общаться с детьми.